# 02 Initial Conditions: Power Spectrum, GRF, and Zel'dovich

This notebook uses the package modules for the linear power spectrum, Gaussian random field generation, and Zel'dovich initial conditions.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("Using src path:", SRC)

In [ ]:
from lcdm_sim.config import load_simulation_config
from lcdm_sim.spectra import power_spectrum
from lcdm_sim.grf import generate_grf
from lcdm_sim.zeldovich import displacement_velocity_fields_from_density, initial_conditions_from_density
import numpy as np

cfg = load_simulation_config(ROOT / "configs" / "smoke.yaml")

In [ ]:
k = np.logspace(-3, 1, 256)
pk = power_spectrum(k, cfg.cosmology, a=cfg.cosmology.a_initial)
print("P(k) finite:", np.isfinite(pk).all(), "min/max:", float(pk.min()), float(pk.max()))

In [ ]:
delta0 = generate_grf(cfg)
disp, vel = displacement_velocity_fields_from_density(delta0, cfg)
state0 = initial_conditions_from_density(delta0, cfg)
print("GRF shape:", delta0.data.shape, "mean/std:", float(delta0.data.mean()), float(delta0.data.std()))
print("Displacement grid:", disp.shape, "Velocity grid:", vel.shape)
print("Particle positions:", state0.positions.shape, "Particle velocities:", state0.velocities.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)
ax[0].imshow(delta0.data[:, :, delta0.n_grid_1d//2].T, origin="lower", cmap="magma")
ax[0].set_title("GRF slice")
pts = state0.positions[:5000]
ax[1].scatter(pts[:,0], pts[:,1], s=1, alpha=0.4)
ax[1].set_title("Initial particles (sample)")
plt.show()